<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Tomás Aguirre
- Nombre de alumno 2: Ignacio Albornoz


## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `https://github.com/tomasaguirre-ignacioalbornoz/MDS7202`

# Importamos librerias útiles

In [7]:
!pip install -qq xgboost optuna
!pip install joblib

# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [8]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y')


df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [9]:
# Importar librerías necesarias
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import joblib


ordered_columns = ['price'] + [col for col in df.columns if col != 'price']
df = df[ordered_columns]

# Crear un FunctionTransformer para extraer día, mes y año
def extract_date_parts(df):
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    df = df.drop(columns=['date'])
    # Crear una lista con 'price' como el primer elemento seguido por el resto de las columnas
    #ordered_columns = ['price'] + [col for col in df.columns if col != 'price']
    #df = df[ordered_columns]
    return df






def get_feature_names(column_transformer):
    """Get feature names from a ColumnTransformer."""
    output_features = []

    for name, pipe, features in column_transformer.transformers_:
        # Process each transformer
        if name != 'remainder':
            if hasattr(pipe, 'get_feature_names_out'):
                # If the transformer has a get_feature_names_out method, use it
                feature_names = pipe.get_feature_names_out(features)
                output_features.extend(feature_names)
            else:
                # Otherwise, just append the feature names as is
                output_features.extend(features)
        else:
            # If the remainder transformer is used, handle accordingly
            remainder_features = [f for f in features if f not in output_features]
            output_features.extend(remainder_features)

    return output_features



# Separar en conjuntos de train, validation y test
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=(1/3), random_state=42)



#print(extract_date_parts(df).head())

date_transformer = FunctionTransformer(extract_date_parts, validate=False)

# Crear un ColumnTransformer para procesar los datos
#numeric_features = df.select_dtypes(include=['int64', 'float64']).columns

numeric_features = df.select_dtypes(include=['int64', 'float64']).drop('quantity', axis=1).columns


# Lista inicial de características categóricas
categorical_features = ['day', 'month', 'year']

# Añadir columnas del DataFrame que no son de tipo int64 o float64
categorical_features.extend(df.select_dtypes(exclude=['int64', 'float64']).columns)

# Excluir la columna 'date'
categorical_features.remove('date')


'''
print("numeric_features")
print(numeric_features)
print("categorical_features")
print(categorical_features)
'''
# Creando un dataframe que solo contiene la columna 'quantity'
train_Y = train_df[['quantity']]

# Creando otro dataframe que contiene todas las columnas excepto 'quantity'
train_X = train_df.drop(columns=['quantity'])

#print(train_X.columns)

# Creando un dataframe que solo contiene la columna 'quantity'
test_Y = test_df[['quantity']]

# Creando otro dataframe que contiene todas las columnas excepto 'quantity'
test_X = test_df.drop(columns=['quantity'])

# Creando un dataframe que solo contiene la columna 'quantity'
val_Y= val_df[['quantity']]

# Creando otro dataframe que contiene todas las columnas excepto 'quantity'
val_X = val_df.drop(columns=['quantity'])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_features),
        ('cat', OneHotEncoder(sparse_output=False), categorical_features)
    ])

'''
# Fit and transform the data
df_eda = preprocessor.fit_transform(df)

# Extract feature names for categorical features transformed by OneHotEncoder
# If 'cat' is the name given to the OneHotEncoder step in your ColumnTransformer
cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out()

print(cat_feature_names)

# Concatenate all feature names (numeric + categorical)
all_feature_names = numeric_features + list(cat_feature_names)

# Create a DataFrame with the new feature names
df_eda = pd.DataFrame(df_eda, columns=all_feature_names)

# Now you can use the corr() method
print(df_eda.corr())
'''

# Crear y entrenar el pipeline con DummyRegressor
pipeline_dummy = Pipeline(steps=[
    ('date', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])


#print("debug1")
pipeline_dummy.fit(train_X, train_Y) 

# Evaluar el modelo
y_pred = pipeline_dummy.predict(val_X)

mae_dummy = mean_absolute_error(val_Y, y_pred)
print(f'MAE con DummyRegressor: {mae_dummy}')


# After fitting your pipeline, call this function
feature_names_after_preprocessing = get_feature_names(pipeline_dummy.named_steps['preprocessor'])
print("Features after preprocessing:", feature_names_after_preprocessing)


# Reemplazar DummyRegressor con XGBRegressor y entrenar nuevamente
pipeline_xgb = Pipeline(steps=[
    ('date', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

pipeline_xgb.fit(train_X, train_Y) 

# Evaluar el nuevo modelo
y_pred_xgb = pipeline_xgb.predict(val_X)
mae_xgb = mean_absolute_error(val_Y, y_pred_xgb)
print(f'MAE con XGBRegressor: {mae_xgb}')

# Guardar los modelos
joblib.dump(pipeline_dummy, 'model_dummy.pkl')
joblib.dump(pipeline_xgb, 'model_xgb.pkl')


MAE con DummyRegressor: 13298.497767341096
Features after preprocessing: ['price', 'id', 'lat', 'long', 'pop', 'day_28', 'day_29', 'day_30', 'day_31', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'year_2012', 'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 'year_2018', 'city_Athens', 'city_Irakleion', 'city_Larisa', 'city_Patra', 'city_Thessaloniki', 'shop_shop_1', 'shop_shop_2', 'shop_shop_3', 'shop_shop_4', 'shop_shop_5', 'shop_shop_6', 'brand_adult-cola', 'brand_gazoza', 'brand_kinder-cola', 'brand_lemon-boost', 'brand_orange-power', 'container_can', 'container_glass', 'container_plastic', 'capacity_1.5lt', 'capacity_330ml', 'capacity_500ml']
MAE con XGBRegressor: 2424.366823499591


['model_xgb.pkl']

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [10]:
# Definir los constraints de monotonía
# Asumiendo que 'price' es la primera columna después del preprocesamiento
# -1 indica una relación monótona negativa
monotone_constraints = (-1,)

# Incluir el inspector de datos en el pipeline antes del regresor
pipeline_xgb_monotone = Pipeline(steps=[
    ('date', date_transformer),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(monotone_constraints=monotone_constraints))
])

pipeline_xgb_monotone.fit(train_X, train_Y)

# Evaluar el modelo
y_pred_xgb_monotone = pipeline_xgb_monotone.predict(val_X)
mae_xgb_monotone = mean_absolute_error(val_Y, y_pred_xgb_monotone)
print(f'MAE con XGBRegressor y constraint monótono: {mae_xgb_monotone}')

# Guardar el modelo
joblib.dump(pipeline_xgb_monotone, 'model_xgb_monotone.pkl')



MAE con XGBRegressor y constraint monótono: 2500.521823322749


['model_xgb_monotone.pkl']

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [11]:
import optuna

# Define la función objetivo para Optuna
def objective(trial):
    # Definir los rangos de búsqueda para los hiperparámetros
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)
    
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    # Crear un nuevo pipeline con los hiperparámetros sugeridos por Optuna
    xgb_model = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda
    )

    # Modificar el valor de min_frequency del OneHotEncoder en el ColumnTransformer
    for name, transformer, columns in preprocessor.transformers_:
        if isinstance(transformer, OneHotEncoder):
            transformer.set_params(min_frequency=min_frequency)

    pipeline_xgb = Pipeline(steps=[
        ('date', date_transformer),
        ('preprocessor', preprocessor),
        ('regressor', xgb_model)
    ])

    pipeline_xgb.fit(train_X, train_Y)

    # Calcular MAE en datos de validación
    y_pred = pipeline_xgb.predict(val_X)
    mae = mean_absolute_error(val_Y, y_pred)

    return mae

# Crear un estudio de Optuna
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=314159))
study.optimize(objective, n_trials=100, timeout=300)

# Obtener los mejores hiperparámetros encontrados
best_params = study.best_params
mae_optuna_1 = study.best_value
num_trials = len(study.trials)

print("Número de trials:", num_trials)
print("Mejores hiperparámetros:", best_params)
print("MAE óptimo:", mae_optuna_1)

Número de trials: 100
Mejores hiperparámetros: {'learning_rate': 0.07030656263631437, 'n_estimators': 829, 'max_depth': 9, 'max_leaves': 89, 'min_child_weight': 3, 'reg_alpha': 0.07298575769548347, 'reg_lambda': 0.7014716927898965, 'min_frequency': 0.6799054775385127}
MAE óptimo: 2034.7008926934


## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [12]:
import optuna
from optuna.integration import XGBoostPruningCallback
from sklearn.base import BaseEstimator, TransformerMixin
optuna.logging.set_verbosity(optuna.logging.WARNING)


def objective_with_prunning(trial):
    # Definir los rangos de búsqueda para los hiperparámetros
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    # Crear el modelo XGBoost con los hiperparámetros sugeridos
    xgb_model = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state=42  # Para garantizar la reproducibilidad
    )

    '''
    # Modificar OneHotEncoder en el preprocesador
    for name, transformer, columns in preprocessor.transformers_:
        if isinstance(transformer, OneHotEncoder):
            transformer.set_params(min_frequency=min_frequency)

    '''

    class ColumnInspector(BaseEstimator, TransformerMixin):
        def fit(self, X, y=None):
            return self

        def transform(self, X):
            # Si X es un DataFrame, imprime las columnas
            if hasattr(X, 'columns'):
                print("Columnas entregadas a XGBoost:", X.columns)
            # Si no, asume que es un array numpy y no hace nada especial
            return X

    # Crear el pipeline
    pipeline_xgb = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', xgb_model)
    ])

    # Aquí aplicamos el preprocesador manualmente para inspeccionar los datos
    train_X_processed = preprocessor.fit_transform(train_X)
    val_X_processed = preprocessor.transform(val_X)

    # Imprimir las columnas después del preprocesamiento
    # Esto asume que el resultado es un DataFrame, si no lo es, ajustar según sea necesario
    if hasattr(train_X_processed, 'columns'):
        print("Columnas entregadas a XGBoost:", train_X_processed.columns)

    # Entrenar el modelo con los datos procesados
    xgb_model.fit(
        train_X_processed, train_Y, 
        eval_set=[(val_X_processed, val_Y)], 
        early_stopping_rounds=100, 
        callbacks=[XGBoostPruningCallback(trial, "validation_0-mae")]
    )

    # Calcular MAE en datos de validación
    y_pred = xgb_model.predict(val_X_processed)
    mae = mean_absolute_error(val_Y, y_pred)

    return mae


study_with_prunning = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=314159))
study_with_prunning.optimize(objective_with_prunning, n_trials=100, timeout=300)

# Obtener los mejores hiperparámetros encontrados con prunning
best_params_with_prunning = study_with_prunning.best_params
mae_optuna_with_prunning = study_with_prunning.best_value
num_trials_with_prunning = len(study_with_prunning.trials)

print("Número de trials con prunning:", num_trials_with_prunning)
print("Mejores hiperparámetros con prunning:", best_params_with_prunning)
print("MAE óptimo con prunning:", mae_optuna_with_prunning)

/home/ignacio/miniconda3/envs/.proyecto1mds/lib/python3.11/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/home/ignacio/miniconda3/envs/.proyecto1mds/lib/python3.11/site-packages/xgboost/sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(
[W 2023-11-15 21:02:52,485] Trial 0 failed with parameters: {'learning_rate': 0.08197440749175473, 'n_estimators': 574, 'max_depth': 6, 'max_leaves': 9, 'min_child_weight': 5, 'reg_alpha': 0.9673564038996015, 'reg_lambda': 0.09820669376309132, 'min_frequency': 0.8018603712796477} because of the following error: KeyError('validation_0-mae').
Traceback (most recent call last):
  File "/home/ignacio/miniconda3/envs/.proyecto1mds/lib/python

KeyError: 'validation_0-mae'

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [ ]:
# Inserte su código acá

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento? 

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [ ]:
# Calcula el MAE para cada modelo
mae_baseline = mae_dummy
mae_xgboost = mae_xgb  
mae_optuna = mae_optuna_1
mae_constraints = mae_xgb_monotone 
mae_pruning = 3.5  

# Crea un DataFrame con los valores de MAE
data = {
    'Modelo': ['Baseline', 'XGBoost', 'XGBoost (Optuna)', 'XGBoost (Constraints)', 'XGBoost (Optuna, Pruning)'],
    'MAE': [mae_baseline, mae_xgboost, mae_optuna, mae_constraints, mae_pruning]
}

tabla = pd.DataFrame(data)

# Imprime la tabla resumen
display(tabla)

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>